I've two dataset saved as updated_intern_data.csv file one having the attributers as aMaterial	RCS (m^2)	Acoustic Reflection Coefficient	Thermal Signature (mW/m^2K)	Density (kg/m?)	Refractive Index	Thermal Conductivity (W/m?K)	Chemical_Symbol

example
Osmium	0.002	0.2	120	22.59	2.25	31.3	Os
Iridium	0.003	0.25	110	22.65	2.2	23	Ir
...

another dataset saved as data2_stable.csv having the combianmtioopn of these compunds witht eh attributes as 
name	entry_id	calculation_id	icsd_id	formationenergy_id	duplicate_entry_id	composition	composition_generic	prototype	spacegroup	volume	ntypes	natoms	unit_cell	sites	band_gap	delta_e	stability	fit	calculation_label	stability_label


example
Ga2Os	17252	21313	103785	4077096	17252	Ga2 Os1	AB2	TiSi2	Fddd	86.6762	2	6	[[0.0, -4.764636, 0.0], [4.156028, -2.382318, 0.0], [0.0, 2.382318, 4.37715]]	['Ga @ 0.332 0.336 0', 'Ga @ 0.668 0.664 0', 'Ga @ 0.418 0.164 0.5', 'Ga @ 0.0822 0.836 0.5', 'Os @ 0 0 0', 'Os @ 0.75 0.5 0.5']	1.035	-0.16094845	0.072465246	standard	static	stable
UGa5Os	27223	21909	600559	4077456	27223	Ga5 Os1 U1	ABC5	CoHoGa5	P4/mmm	127.628	3	7	[[4.362245, 0.0, 0.0], [0.0, 4.362245, 0.0], [0.0, 0.0, 6.706968]]	['Ga @ 0.5 0.5 0', 'Ga @ 0.5 0 0.301', 'Ga @ 0 0.5 0.301', 'Ga @ 0.5 0 0.699', 'Ga @ 0 0.5 0.699', 'Os @ 0 0 0.5', 'U @ 0 0 0']	0	-0.275006081	0	standard	static	unstable
PuOs2	17901	22191	105559	4077750	17901	Os2 Pu1	AB2	Cu2Mg	Fd-3m	103.783	2	6	[[3.729918, -3.729918, 0.0], [-3.729918, 0.0, -3.729918], [3.729918, 3.729918, 0.0]]	['Os @ 0.375 0.75 0.125', 'Os @ 0.375 0.25 0.625', 'Os @ 0.875 0.25 0.625', 'Os @ 0.375 0.75 0.625', 'Pu @ 0 0 0', 'Pu @ 0.75 0.5 0.25']	0	0.38827297	0.405408951	standard	static	stable
.....

using these two dataset i want to build a project used to detect whethe the generated compunds can be use for stealth or using the parameters
 Band gap: 1-5 eV, 
 Delta_e: 0.1-0.5 eV 

so dveelopa tensorfloe model as follows
- Create a Generative Adversarial Network (GAN) model to generate new data for compounds using data2_stable.csv dataset .
- a model to predict properties of the generated compounds.
- Based on the predicted properties, decide whether the compound can be used for stealth or not.

the geenrated compund would be in tensor or arraty form ryt create a mapping fucntionw hcih will map back the compunds, use updated_intern_data.csv- Chemical_Symbol attrivute to mapp

- Create a Generative Adversarial Network (GAN) model to generate new data for compounds.
- Use a model to predict properties of the generated compounds.
- Based on the predicted properties, decide whether the compound can be used for stealth or not.

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras

# Load the data
data = pd.read_csv('data2_stable.csv')
data_updated = pd.read_csv('updated_intern_data.csv')

# Drop irrelevant columns
cols_to_drop = ['entry_id', 'calculation_id', 'icsd_id', 'formationenergy_id', 'duplicate_entry_id', 'spacegroup', 'fit', 'unit_cell', 'sites']
data = data.drop(columns=cols_to_drop)

# One-Hot encode the 'name' and other categorical columns
encoder = OneHotEncoder(sparse=False)
encoded_features = encoder.fit_transform(data[['name', 'composition', 'composition_generic', 'prototype', 'calculation_label', 'stability_label']])
encoded_df = pd.DataFrame(encoded_features)

# Normalize continuous columns
scaler = MinMaxScaler()
data[['volume', 'ntypes', 'natoms', 'band_gap', 'delta_e', 'stability']] = scaler.fit_transform(data[['volume', 'ntypes', 'natoms', 'band_gap', 'delta_e', 'stability']])

# Concatenate encoded features with the main dataset
data = pd.concat([data, encoded_df], axis=1).drop(columns=['name', 'composition', 'composition_generic', 'prototype', 'calculation_label', 'stability_label'])


c:\Users\Asus\anaconda3\envs\Capstone_ml\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## GAN Model

In [45]:
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization, Reshape, Flatten
from tensorflow.keras.models import Model

# Generator
def build_generator():
    model = keras.Sequential()
    
    model.add(Dense(128, input_dim=100))  # Input is random noise
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(Dense(np.prod(data.shape[1]), activation='sigmoid'))  # Adjust this based on data's shape
    model.add(Reshape((data.shape[1],)))
    
    return model

# Discriminator
def build_discriminator():
    model = keras.Sequential()
    
    model.add(Flatten(input_shape=(data.shape[1],)))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

generator = build_generator()
discriminator = build_discriminator()

discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [41]:
X_train

volume  ntypes    natoms  stability    0    1    2    3    4    5  \
911   0.306201    0.50  0.341772   0.010602  0.0  0.0  0.0  0.0  0.0  0.0   
355   0.028260    0.50  0.037975   0.000000  0.0  0.0  0.0  0.0  0.0  0.0   
382   0.141484    0.25  0.037975   0.014934  0.0  0.0  0.0  0.0  0.0  0.0   
238   0.047437    0.25  0.037975   0.021981  0.0  0.0  0.0  0.0  0.0  0.0   
1419  0.063664    0.25  0.037975   0.000024  0.0  0.0  0.0  0.0  0.0  0.0   
...        ...     ...       ...        ...  ...  ...  ...  ...  ...  ...   
976   0.169893    0.25  0.113924   0.014523  0.0  0.0  0.0  0.0  0.0  0.0   
393   0.040927    0.25  0.037975   0.000390  0.0  0.0  0.0  0.0  0.0  0.0   
1464  0.027807    0.50  0.037975   0.000000  0.0  0.0  0.0  0.0  0.0  0.0   
1469  0.030681    0.25  0.037975   0.003434  0.0  0.0  0.0  0.0  0.0  0.0   
742   0.109614    0.25  0.037975   0.097567  0.0  0.0  0.0  0.0  0.0  0.0   

      ...  2633  2634  2635  2636  2637  2638  2639  2640  2641  2642  
911   ...   0.0   0.0   0.0   0.0   0.0   1.0   1.0   0.0   0.0   1.0  
355   ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   1.0  
382   ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   1.0   0.0  
238   ...   0.0   0.0   0.0   0.0   0.0   1.0   0.0   1.0   1.0   0.0  
1419  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   1.0  
...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  
976   ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   1.0  
393   ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   1.0  
1464  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   1.0  
1469  ...   0.0   0.0   0.0   0.0   0.0   1.0   0.0   1.0   0.0   1.0  
742   ...   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   1.0   0.0  

[1240 rows x 2647 columns]

In [149]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Dropout, Reshape, Flatten,ReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2

'''
def build_generator():
    model = Sequential()
    
    model.add(Dense(128, input_dim=150))
    model.add(LeakyReLU(0.002987))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Dropout(0.002912))
    model.add(Dense(256))
    model.add(LeakyReLU(0.001004532))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dropout(0.02))
    model.add(Dense(512))
    model.add(LeakyReLU(0.039834))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Dropout(0.038))
    model.add(Dense(1024))
    model.add(LeakyReLU(0.0093129))
    model.add(Dropout(0.4))
    model.add(Dense(np.prod(data.shape[1]), activation='sigmoid'))
    model.add(Reshape((data.shape[1],)))
    
    return model
def build_discriminator():
    model = Sequential()
    
    model.add(Flatten(input_shape=(data.shape[1],)))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.002))
    model.add(Dropout(0.4))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.01982))
    model.add(Dropout(0.4))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.00296))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.032344))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    
    return model
'''
def build_generator():
    model = Sequential()

    model.add(Flatten(input_shape=(data.shape[1],)))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.002))
    model.add(Dropout(0.4))
    
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Dropout(0.3))
    
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Dropout(0.25))
    
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Dropout(0.2))

    model.add(Dense(1, activation='sigmoid'))
    return model


def build_discriminator():
    model = Sequential()

    # Input layer
    model.add(Flatten(input_shape=(data.shape[1],)))
    
    # Hidden layers
    model.add(Dense(1024, kernel_regularizer=l2(0.01)))
    model.add(LeakyReLU(alpha=0.002))
    model.add(Dropout(0.4))
    
    model.add(Dense(512, kernel_regularizer=l2(0.01)))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Dropout(0.3))
    
    model.add(Dense(256, kernel_regularizer=l2(0.01)))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Dropout(0.25))
    
    model.add(Dense(128, kernel_regularizer=l2(0.01)))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Dropout(0.2))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    return model

def build_combined():
    discriminator.trainable = False

    model = Sequential()
    model.add(generator)
    model.add(discriminator)

    return model

optimizer = keras.optimizers.Adam(0.000019, 0.79)

discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0002, clipvalue=1.0, decay=1e-8)
generator_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=discriminator_optimizer, metrics=['accuracy'])

generator = build_generator()
combined = build_combined()
combined.compile(loss='binary_crossentropy', optimizer=generator_optimizer)

def train_gan(epochs, batch_size=128, save_interval=50):
    X_train = data.values

    valid_smooth = np.ones((batch_size, 1)) * 0.999
    fake_smooth = np.zeros((batch_size, 1)) * 0.001

    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        
        noise = np.random.normal(0, 1, (batch_size, 150))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, valid_smooth)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake_smooth)
        d_loss = 0.05* np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, (batch_size, 150))  # Changed to 150 from 100
        g_loss = combined.train_on_batch(noise, valid_smooth)

        print(f"{epoch}/{epochs} [D loss: {d_loss[0]} | D accuracy: {d_loss[1]}] [G loss: {g_loss}]")
train_gan(epochs = 30)

c:\Users\Asus\anaconda3\envs\Capstone_ml\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


ValueError: Exception encountered when calling layer "sequential_202" (type Sequential).

Input 0 of layer "dense_766" is incompatible with the layer: expected axis -1 of input shape to have value 2649, but received input with shape (None, 1)

Call arguments received by layer "sequential_202" (type Sequential):
  • inputs=tf.Tensor(shape=(None, 1), dtype=float32)
  • training=False
  • mask=None

## Traing GAN Model

In [99]:
z = Input(shape=(100,))
img = generator(z)

discriminator.trainable = False

valid = discriminator(img)

combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer='adam')

def train_gan(epochs, batch_size=128, save_interval=50):
    # Training data
    X_train = data.values

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Training the discriminator
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        input_dim = 150
        
        noise = np.random.normal(0, 1, (batch_size, input_dim))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Training the generator
        noise = np.random.normal(0, 1, (batch_size, input_dim))
        g_loss = combined.train_on_batch(noise, valid)

        print(f"{epoch}/{epochs} [D loss: {d_loss[0]} | D accuracy: {d_loss[1]}] [G loss: {g_loss}]")

        if epoch % save_interval == 0:
            # You can save generated samples here for inspection
            pass

train_gan(epochs=50)


ValueError: Exception encountered when calling layer "sequential_19" (type Sequential).

Input 0 of layer "dense_72" is incompatible with the layer: expected axis -1 of input shape to have value 150, but received input with shape (None, 100)

Call arguments received by layer "sequential_19" (type Sequential):
  • inputs=tf.Tensor(shape=(None, 100), dtype=float32)
  • training=False
  • mask=None

## Predictive Model

In [6]:
X = data.drop(columns=['band_gap', 'delta_e'])
y = data[['band_gap', 'delta_e']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

predictor = keras.Sequential()
predictor.add(Dense(512, activation='relu', input_dim=X_train.shape[1]))
predictor.add(Dense(256, activation='relu'))
predictor.add(Dense(2, activation='linear'))  # Predicting band_gap and delta_e

predictor.compile(optimizer='adam', loss='mse')
predictor.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/100
39/39 [==============================] - 1s 17ms/step - loss: nan - val_loss: nan
Epoch 2/100
39/39 [==============================] - 0s 11ms/step - loss: nan - val_loss: nan
Epoch 3/100
39/39 [==============================] - 0s 10ms/step - loss: nan - val_loss: nan
Epoch 4/100
39/39 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 5/100
39/39 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 6/100
39/39 [==============================] - 0s 7ms/step - loss: nan - val_loss: nan
Epoch 7/100
39/39 [==============================] - 0s 7ms/step - loss: nan - val_loss: nan
Epoch 8/100
39/39 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 9/100
39/39 [==============================] - 0s 7ms/step - loss: nan - val_loss: nan
Epoch 10/100
39/39 [==============================] - 0s 8ms/step - loss: nan - val_loss: nan
Epoch 11/100
39/39 [==============================] - 0s 7ms/step 

In [7]:
def is_stealthy(compound):
    band_gap, delta_e = predictor.predict(np.array([compound]))
    if 1 <= band_gap <= 5 and 0.1 <= delta_e <= 0.5:
        return True
    return False

# Test on a generated compound
compound = generator.predict(np.random.normal(0, 1, (1, 100)))
print(is_stealthy(compound))
if is_stealthy(compound):
    print("The generated compound can be used for stealth.")
else:
    print("The generated compound cannot be used for stealth.")

1/1 [==============================] - 0s 202ms/step


ValueError: in user code:

    File "c:\Users\Asus\anaconda3\envs\Capstone_ml\lib\site-packages\keras\engine\training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Asus\anaconda3\envs\Capstone_ml\lib\site-packages\keras\engine\training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Asus\anaconda3\envs\Capstone_ml\lib\site-packages\keras\engine\training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Asus\anaconda3\envs\Capstone_ml\lib\site-packages\keras\engine\training.py", line 1791, in predict_step
        return self(x, training=False)
    File "c:\Users\Asus\anaconda3\envs\Capstone_ml\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Asus\anaconda3\envs\Capstone_ml\lib\site-packages\keras\engine\input_